In [ ]:
# Imports for InfluxdbV2
from utilities import ExecutionTime, list_from_file
from influxdb_client import Point, InfluxDBClient, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import pendulum

# Imports for OpenOPC2
from openopc2.da_client import OpcDaClient, OPCError
from openopc2.gateway_proxy import OpenOpcGatewayProxy
from openopc2.config import OpenOpcConfig
from Pyro5.errors import ProtocolError
from options import optobj

import time
from rich.console import Console
console = Console() # Create rich console

In [ ]:
"""
Connect to InfluxDB 2.0.9 - write data
Configure credentials and Parameters for OpenOPCV2
"""

console = Console() # Create rich console
influx_cloud_url = 'http://192.168.1.87:8086/'
influx_cloud_token = 'f0psEpdC89AllAEweF4FS3d3GQzBw-EkHT4tOhOHh3U2f-pBLZkn8FfabsjsdKAWM2NwbQn7XxPvltB1U-xctQ=='
bucket = 'OPC'
org = 'home'

options = optobj
# options.svr = 'OPC.DeltaV.1'
options.svr = 'Matrikon.OPC.Simulation.1'
options.ip = '192.168.1.2'
options.mode = 'gateway'  # TODO: Update Main Options with new parameter mode,  remove local
options.sleepTime = 1
options.timeout = 5000 * 4
options.sliceamt = 1000

config = OpenOpcConfig()
config.OPC_SERVER = "Matrikon.OPC.Simulation" if options.svr is None else options.svr
config.OPC_GATEWAY_HOST = "192.168.1.10" if options.ip is None else options.ip
config.OPC_CLASS = "Graybox.OPC.DAWrapper"
config.OPC_MODE = options.mode

# Get Matrikon Simulator Tags for reading.
tags = list_from_file('tags.txt')
group = 'Simulation'

console.print(tags)

In [ ]:
if config.OPC_MODE == "gateway":
    opc = OpenOpcGatewayProxy(config.OPC_GATEWAY_HOST, config.OPC_GATEWAY_PORT).get_opc_da_client_proxy()
    console.print("Connecting in Gateway Mode")
else:
    opc = OpcDaClient(config)
    console.print("Connecting in COM Mode")

opc.connect(config.OPC_SERVER, config.OPC_HOST)

In [ ]:
with InfluxDBClient(url=influx_cloud_url, token=influx_cloud_token) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)

    while True:
        data = opc.read(tags)
        points = []
        for point in data:
            d = {
                "measurement": "data",  # Group
                "tags": {"Site": "PA1"},  # "path": opc path
                "fields": {point[0]: point[1]}, # Tagname : Data
                "time": point[3]  # OPC Time Field
            }
            points.append(Point.from_dict(d, WritePrecision.NS))  
    
        """
        Influxdb v2.0.9 Write Data.
        """
        timer = ExecutionTime()  # Start the timer
        write_api.write(bucket, org, points)
        console.print(f"Influxdb write was performed in {timer.human()}!!")
        data = None

        time.sleep(2)


    

In [55]:
opc.remove(group)
opc.close()